# Airline flight delay dataset

This notebook looks to assess the uncertainty estimates of Mondrian Forests on a (albeit simple) regression problem.  The data consists of flight delays, with 8 features as predictors.  This is briefly described in [the paper](http://www.gatsby.ucl.ac.uk/~balaji/mfregression_aistats16.pdf).

I find that the MSE that models report below agree qutie well with the paper.  The main goal was to assess uncertainty estimates in a more interpretible way than presented in the paper.  I find that, indeed, the uncertainty estimates for this particular problem do align well with expection from Gaussian statistics.

## Load the flight data

This data was originally cleaned by by James Hensman, and lives in a cryptic site mirror thanks to Neil Lawrence (yay semi-open science! (not actually sarcastic)).  The data is a pickled pandas DF.

In [29]:
import numpy as np
import pandas as pd

# this pickle file can be found as 'filtered_data.pickle' at
# this url: http://staffwww.dcs.shef.ac.uk/people/N.Lawrence/dataset_mirror/airline_delay/
data = pd.read_pickle('./process_data/airline-delays/airline-delays.p')

num_train = 7000
num_test = 1000

# WARNING: removing year
data.pop('Year')

# Get data matrices
Yall = data.pop('ArrDelay').values[:,None]
Xall = data.values

# Subset the data (memory!!)
all_data = num_train+num_test
Xall = Xall[:all_data]
Yall = Yall[:all_data]

np.random.seed(seed=1234)
N_shuffled = np.random.permutation(Yall.shape[0])
train, test = N_shuffled[num_test:], N_shuffled[:num_test]

X_train, X_test, y_train, y_test = Xall[train], Xall[test], Yall[train], Yall[test]

This next cell creates a dict that the MF code plays nice with.

In [30]:
def make_data(xtrain, xtest, ytrain, ytest):
    data = {}
    data['n_dim'] = xtrain.shape[1]
    data['x_train'] = xtrain
    data['y_train'] = ytrain.ravel()
    data['n_train'] = xtrain.shape[0]
    data['x_test'] = xtest
    data['y_test'] = ytest.ravel()
    data['n_test'] = xtest.shape[0]
    data['n_class'] = 1
    data['is_sparse'] = 0
    data['train_ids_partition'] = {}
    
    data['train_ids_partition']['current'] = {0:np.arange(xtrain.shape[0])}
    data['train_ids_partition']['cumulative'] = {0:np.arange(xtrain.shape[0])}
    return data
data = make_data(X_train, X_test, y_train, y_test)
print X_train.shape

(7000, 8)


# Models for comparison

For benchmarking purposes, run RF and ERT models.

In [31]:
from time import time
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

np.random.seed(12345)
def run_regressor(rgr, data):
    """
    Run a sklearn regressor, return time and score
    """
    t0 = time()
    rgr.fit(data['x_train'].copy(), data['y_train'].copy())
    score = mean_squared_error(rgr.predict(data['x_test'].copy()), data['y_test'].copy())
    run_time = time()-t0
    return run_time, score

rf_time, rf_score = run_regressor(RandomForestRegressor(10, min_samples_leaf=5), data)
et_time, et_score = run_regressor(ExtraTreesRegressor(10, min_samples_leaf=5), data)

## Punking the command line

I do not personally enjoy the command line interface of the MF code.  Below is a class containing the settings necessary to run the code.

In [32]:
class ParamSettings(object):
    def __init__(self):
        self.dataset = 'toy'
        self.normalize_features = 1
        self.select_features = 0
        self.optype = 'real'
        self.data_path = './process_data/'
        self.debug = 0
        self.op_dir = 'results'
        self.tag = ''
        self.save = 0
        self.verbose = 1
        self.init_id = 1
        self.n_mondrians = 10
        self.budget = -1 # -1 sets lifetime to inf
        self.discount_factor = 10
        self.n_minibatches = 1
        self.draw_mondrian = 0
        self.smooth_hierarchically = 0 # the code doesnt seem to play nice for regression with this on
        self.store_every = 0
        self.bagging = 0
        self.min_samples_split = 10 # this is the rec in the paper
        self.name_metric = 'mse'
        
        if self.optype == 'class':
            self.alpha = 0    # normalized stable prior
            assert self.smooth_hierarchically
            
        if self.budget < 0:
            self.budget_to_use = np.inf
        else:
            self.budget_to_use = settings.budget
        
settings = ParamSettings()

In [35]:
from src.mondrianforest_utils import load_data, reset_random_seed, precompute_minimal
from src.mondrianforest import process_command_line, MondrianForest
np.random.seed(12345)

# data loading and cache
param, cache = precompute_minimal(data, settings)

mf = MondrianForest(settings, data)
print '\nminibatch\tmetric_train\tmetric_test\tnum_leaves'
t0=time()

# loop over minibatches
for idx_minibatch in range(settings.n_minibatches):
    train_ids_current_minibatch = data['train_ids_partition']['current'][idx_minibatch]
    if idx_minibatch == 0:
        # Batch training for first minibatch
        mf.fit(data, train_ids_current_minibatch, settings, param, cache)
    else:
        # Online update
        mf.partial_fit(data, train_ids_current_minibatch, settings, param, cache)

    # produce predictions
    weights_prediction = np.ones(settings.n_mondrians) * 1.0 / settings.n_mondrians
    train_ids_cumulative = data['train_ids_partition']['cumulative'][idx_minibatch]
    pred_forest_train, metrics_train = \
        mf.evaluate_predictions(data, data['x_train'][train_ids_cumulative, :], \
        data['y_train'][train_ids_cumulative], \
        settings, param, weights_prediction, False)
    pred_forest_test, metrics_test = \
        mf.evaluate_predictions(data, data['x_test'], data['y_test'], \
        settings, param, weights_prediction, False)
    name_metric = settings.name_metric     # acc or mse
    metric_train = metrics_train[name_metric]
    metric_test = metrics_test[name_metric]
    tree_numleaves = np.zeros(settings.n_mondrians)
    for i_t, tree in enumerate(mf.forest):
        tree_numleaves[i_t] = len(tree.leaf_nodes)
    forest_numleaves = np.mean(tree_numleaves)
    print '%9d\t%.3f\t\t%.3f\t\t%.3f' % (idx_minibatch, metric_train, metric_test, forest_numleaves)
mf_time = time() - t0



minibatch	metric_train	metric_test	num_leaves
log_prob (using Gaussian approximation) = -4.118255
log_prob (using mixture of Gaussians) = -3.558153
log_prob (using Gaussian approximation) = -4.730813
log_prob (using mixture of Gaussians) = -4.311993
        0	505.538		633.025		1479.700


# Prediction summary

Below here we compare the point estimate predictions, and the time taken to produce them.

In [37]:
print 'RF time, rmse = %0.2f, %0.2f' % (rf_time, np.sqrt(rf_score))
print 'ET time, rmse = %0.2f, %0.2f' % (et_time, np.sqrt(et_score))
print 'MF time, rmse = %0.2f, %0.2f' % (mf_time, np.sqrt(metric_test))

RF time, rmse = 0.30, 23.97
ET time, rmse = 0.11, 24.02
MF time, rmse = 7.87, 25.16


# Uncertainty estimates

Below here we test if the uncertainty estimates are reasonable. Indeed they are, falling very close to the expected ideal for a Gaussian distribution. 

In [34]:
chi = np.abs(data['y_test'].ravel() - pred_forest_test['pred_mean']) / np.sqrt(pred_forest_test['pred_var']) 

print 'Percentage within 1sigma = %0.2f' % (1. * np.where(chi < 1)[0].size / data['y_test'].size)
print 'Percentage within 2sigma = %0.2f' % (1. * np.where(chi < 2)[0].size / data['y_test'].size)

Percentage within 1sigma = 0.80
Percentage within 2sigma = 0.95
